<h1>Neighborhood Min Distance (NMD) descriptor routines</h1>
<h1>get_batch_nmd</h1>
This function calculates the Neighborhood Min Distance (NMD) descriptor from two batches of images. This function is used by the get_nmd function for memory effeciency if the data is too large.

<h3>Parameters</h3>

<h5>P</h5>
A 4D numpy array containing the batch of facial images of parents.
<br><b>Shape:</b> (NumberOfImages, Height, Width, NumberOfChannels)

<h5>C</h5>
A 4D numpy array containing the batch of facial images of children.
<br><b>Shape:</b> (NumberOfImages, Height, Width, NumberOfChannels)

<h5>N</h5>
The width and height (size) of the NMD patch (neighborhood).

<h3>Return Value</h3>

A 2D numpy array containing the NMD descriptors of all pairs from batches P and C.
<br><b>Shape:</b> (NumberOfImages, (Height-N+1) &#215; (Width-N+1))

<h3>Notes</h3>
The NumberOfImages, Height, Width, and NumberOfChannels of parents and children images must be identical to each other.

<h1>get_nmd</h1>
This function calculates the Neighborhood Min Distance (NMD) descriptor from two sets of images.

<h3>Parameters</h3>

<h5>P</h5>
A 4D numpy array containing the set of facial images of parents.
<br><b>Shape:</b> (NumberOfImages, Height, Width, NumberOfChannels)

<h5>C</h5>
A 4D numpy array containing the set of facial images of children.
<br><b>Shape:</b> (NumberOfImages, Height, Width, NumberOfChannels)

<h5>N</h5>
The width and height (size) of the NMD patch (neighborhood).

<h3>Return Value</h3>

A 2D numpy array containing the NMD descriptors of all pairs from sets P and C.
<br><b>Shape:</b> (NumberOfImages, (Height-N+1) &#215; (Width-N+1))

<h3>Notes</h3>
The NumberOfImages, Height, Width, and NumberOfChannels of parents and children images must be identical to each other.

In [1]:
import numpy as np

batch_size = 100

def get_batch_nmd(P, C, N):
    h = N // 2
    assert P.shape == C.shape
    CropP = P[:, h:-h, h:-h, :]
    Dists = np.empty((N*N, P.shape[0], P.shape[1]-N+1, P.shape[2]-N+1))
    p = 0
    for i in range(-h, h+1):
        for j in range(-h, h+1):
            i0, i1 = i+h, C.shape[1]+i-h
            j0, j1 = j+h, C.shape[2]+j-h
            CropC = C[:, i0:i1, j0:j1, :]
            Dists[p, :, :, :] = np.sum(np.square(CropP-CropC), axis=3)
            p = p + 1
    NMD3D = np.min(Dists, axis=0)
    NMD = np.empty((NMD3D.shape[0],NMD3D.shape[1]*NMD3D.shape[2]))
    for p in range(NMD3D.shape[0]):
        NMD[p, :] = NMD3D[p].flatten()
    return NMD

def get_nmd(P, C, N):
    assert P.shape == C.shape
    NMD = np.empty((P.shape[0],(P.shape[1]-N+1)*(P.shape[2]-N+1)))
    for batch in range(0, P.shape[0], batch_size):
        size = min(batch_size, P.shape[0]-batch)
        NMD[batch:batch+size, :] = get_batch_nmd(P[batch:batch+size, :, :, :], C[batch:batch+size, :, :, :], N)
    return NMD

<h1>Kinship data loading routines</h1>
<h1>load_txt_pairs</h1>
Loads kinship pairs from a text file following our encoding in which each line in the text file corresponds to a single pair and has the following format:

fold_number kinship_label parent_image_file_name child_image_file_name

<h3>Parameters</h3>

<h5>filename</h5>
A string indicating the path to the pairs text file.

<h3>Return Value</h3>

A list of pairs, each pair is a tuple of four elements corresponding to:

<ol>
    <li> The fold number </li>
    <li> The kinship label </li>
    <li> The file name of the parent image </li>
    <li> The file name of the child image </li>
</ol>

<h1>load_mat_pairs</h1>
Loads kinship pairs from a matlab variable file following KinFaceW schema.

<h3>Parameters</h3>

<h5>filename</h5>
A string indicating the path to the pairs matlab variable file.

<h3>Return Value</h3>

A list of pairs, each pair is a tuple of four elements corresponding to:

<ol>
    <li> The fold number </li>
    <li> The kinship label </li>
    <li> The file name of the parent image </li>
    <li> The file name of the child image </li>
</ol>

<h1>get_image_dirs</h1>
Gives the directories containing the facial images of parents and children.

<h3>Parameters</h3>

<h5>rootdir</h5>
A string indicating the path of the directory containing all kinship datasets.

<h5>dataset</h5>
A string indicating the name of the kinship dataset, must be one of these values: 'KinFaceW-I', 'KinFaceW-II', or 'CornellKinFace'.

<h5>subset</h5>
A string indicating the name of the kinship subset, unused with the CornellKinFace dataset, must be one of these values with the KinFaceW datasets: 'fs', 'fd', 'ms', or 'md'.

<h3>Return Value</h3>

A tuple containing the pathes of the parents image-directory and the children image-directory.

<h1>load_pairs</h1>
Loads the kinship pairs from the meta file of a dataset.

<h3>Parameters</h3>

<h5>rootdir</h5>
A string indicating the path of the directory containing all kinship datasets.

<h5>dataset</h5>
A string indicating the name of the kinship dataset, must be one of these values: 'KinFaceW-I', 'KinFaceW-II', or 'CornellKinFace'.

<h5>subset</h5>
A string indicating the name of the kinship subset, unused with the CornellKinFace dataset, must be one of these values with the KinFaceW datasets: 'fs', 'fd', 'ms', or 'md'.

<h3>Return Value</h3>

A list of pairs, each pair is a tuple of four elements corresponding to:

<ol>
    <li> The fold number </li>
    <li> The kinship label </li>
    <li> The file name of the parent image </li>
    <li> The file name of the child image </li>
</ol>

<h1>load_fold</h1>
Loads the kinship data from one fold of a certain dataset.

<h3>Parameters</h3>

<h5>rootdir</h5>
A string indicating the path of the directory containing all kinship datasets.

<h5>dataset</h5>
A string indicating the name of the kinship dataset, must be one of these values: 'KinFaceW-I', 'KinFaceW-II', or 'CornellKinFace'.

<h5>subset</h5>
A string indicating the name of the kinship subset, unused with the CornellKinFace dataset, must be one of these values with the KinFaceW datasets: 'fs', 'fd', 'ms', or 'md'.

<h5>fold</h5>
The number of fold to load depending on the meta-data (list of pairs) of the dataset.

<h3>Return Value</h3>

A tuple containing the following:

<ol>
    <li> 
        A tuple containing the training data: 
        <ol>
            <li> <b>The parents images:</b> A numpy array of shape (NumberOfImages, Height, Width, NumberOfChannels) </li>
            <li> <b>The children images:</b> A numpy array of shape (NumberOfImages, Height, Width, NumberOfChannels) </li>
            <li> <b>The kinship labels:</b> A numpy array of shape (NumberOfImages, 1) </li>
        </ol>
    </li>
     <li> 
        A tuple containing the test data: 
        <ol>
            <ol>
            <li> <b>The parents images:</b> A numpy array of shape (NumberOfImages, Height, Width, NumberOfChannels) </li>
            <li> <b>The children images:</b> A numpy array of shape (NumberOfImages, Height, Width, NumberOfChannels) </li>
            <li> <b>The kinship labels:</b> A numpy array of shape (NumberOfImages, 1) </li>
        </ol>
        </ol>
    </li>
</ol>

In [10]:
from imageio import imread
import scipy.io as sio
import numpy as np
from os.path import join

def load_txt_pairs(filename):
    metafile = open(filename, 'r')
    metadata = metafile.read()
    metafile.close()
    metalines = [line for line in metadata.split('\n') if len(line)>0]
    str_pairs = [metaline.split(' ') for metaline in metalines]
    pairs = [[int(pair[0]), int(pair[1]), pair[2], pair[3]] for pair in str_pairs]
    return pairs

def load_mat_pairs(filename):
    meta = sio.loadmat(filename)
    pairs = []
    for p in meta['pairs']:
        pairs.append([p[0][0][0], p[1][0][0], p[2][0], p[3][0]])
    return pairs

def get_image_dirs(rootdir, dataset, subset):
    PrefixToDir={'fd':'father-dau','fs':'father-son','md':'mother-dau','ms':'mother-son'}
    if dataset[:10]=='KinFaceW-I':
        same_dir = join(rootdir, dataset, 'images', PrefixToDir[subset])
        return (same_dir, same_dir)
    if dataset=='CornellKinFace':
        parents_dir = join(rootdir, dataset, 'Parents')
        children_dir = join(rootdir, dataset, 'Children')
        return (parents_dir, children_dir)
    return None

def load_pairs(rootdir, dataset, subset):
    if dataset=='KinFaceW-I' or dataset=='KinFaceW-II':
        return load_mat_pairs(join(rootdir, dataset, 'meta_data', subset + '_pairs.mat'))
    if dataset=='CornellKinFace':
        return load_txt_pairs(join(rootdir, dataset, 'cornell-meta.txt'))
    return None

def load_fold(rootdir, dataset, subset, fold):
    pairs = load_pairs(rootdir, dataset, subset)
    image_dirs = get_image_dirs(rootdir, dataset, subset)
    P0, C0, K0 = [], [], []
    P1, C1, K1 = [], [], []
    for p in pairs:
        pImg = imread(join(image_dirs[0], p[2])) / 255
        cImg = imread(join(image_dirs[1], p[3])) / 255
        if p[0] == fold:
            P1.append(pImg)
            C1.append(cImg)
            K1.append(p[1])
        else:
            P0.append(pImg)
            C0.append(cImg)
            K0.append(p[1])
    return ((np.array(P0), np.array(C0), np.array(K0)), (np.array(P1), np.array(C1), np.array(K1)))

<h1> The NMD experiments</h1>
You must change line #4 indicating the path to the directory containing all kinship datasets.

Line #6 specifies the different datasets to experiment on.

Line #16 specifies the SVM grid search parameters.

Lines #17 and #18 specify the Random Forest grid search parameters.

Lines #26 and #27 calculate the training and test NMD descriptors, you may want to change the third parameter indicating the NMD patch-size.

<h3> Notes </h3>
Because the <b>Random Forest</b> classifier has an incorporated random process in it, the results may change each time it's executed. To overcome this you can add a loop that performs serveral trials for each fold and report the best results.

In [14]:
from sklearn import svm
from sklearn import ensemble

rootdir = 'D:/PhD/DataSets'

datasets = [('CornellKinFace', 'all', 5),
            ('KinFaceW-I', 'fs', 5),
            ('KinFaceW-I', 'fd', 5),
            ('KinFaceW-I', 'ms', 5),
            ('KinFaceW-I', 'md', 5),
            ('KinFaceW-II', 'fs', 5),
            ('KinFaceW-II', 'fd', 5),
            ('KinFaceW-II', 'ms', 5),
            ('KinFaceW-II', 'md', 5)]

nu_values = np.arange(0.05, 1.0, 0.05)
max_depth_values = [max_depth for max_depth in range(10, 16)]
n_estimators_values = [n_estimators for n_estimators in range(1, 6)]
for (dataset, kinship, n_fold) in datasets:
    print(dataset, kinship)
    svm_accu_grid = np.zeros(len(nu_values), dtype=np.float32)
    rdf_accu_grid = np.zeros((len(max_depth_values), len(n_estimators_values)), dtype=np.float32)
    for fold in range(n_fold):
        ((P0, C0, K0), (P1, C1, K1)) = load_fold(rootdir, dataset, kinship, fold+1)

        NMD0 = get_nmd(P0, C0, 15)
        NMD1 = get_nmd(P1, C1, 15)


        for i, nu in enumerate(nu_values):
            model = svm.NuSVC(nu=nu, kernel="rbf", verbose=False, gamma="auto")
            model.fit(NMD0, K0)
            accuracy = model.score(NMD1, K1)
            svm_accu_grid[i] = svm_accu_grid[i] + accuracy
            
        for i, max_depth in enumerate(max_depth_values):
            for j, n_estimators in enumerate(n_estimators_values):
                best_accu = 0.0
                for _ in range(5):
                    model = ensemble.RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators, max_features=1)
                    model.fit(NMD0, K0)
                    accuracy = model.score(NMD1, K1)
                    if accuracy > best_accu:
                        best_accu = accuracy
                rdf_accu_grid[i, j] = rdf_accu_grid[i, j] + best_accu

    best_nu = 0
    best_accu = 0
    for i, nu in enumerate(nu_values):
        if svm_accu_grid[i] > best_accu:
            best_accu = svm_accu_grid[i]
            best_nu = nu
    
    print('SVM: %.02f%% [nu: %g]'%(100*best_accu/n_fold, best_nu))
    
    best_d = 0
    best_n = 0
    best_a = 0
    for i, max_depth in enumerate(max_depth_values):
        for j, n_estimators in enumerate(n_estimators_values):
            if rdf_accu_grid[i, j]>best_a:
                best_d = max_depth
                best_n = n_estimators
                best_a = rdf_accu_grid[i, j]
    
    print('RandomForest: %.02f%% [max_depth: %d] [n_estimators: %d]'%(100*best_a/n_fold, best_d, best_n))

CornellKinFace all
SVM: 71.58% [nu: 0.4]
RandomForest: 77.97% [max_depth: 13] [n_estimators: 5]
KinFaceW-I fs
SVM: 67.30% [nu: 0.7]
RandomForest: 72.14% [max_depth: 11] [n_estimators: 5]
KinFaceW-I fd
SVM: 59.36% [nu: 0.8]
RandomForest: 69.03% [max_depth: 11] [n_estimators: 4]
KinFaceW-I ms
SVM: 65.09% [nu: 0.55]
RandomForest: 75.02% [max_depth: 12] [n_estimators: 5]
KinFaceW-I md
SVM: 72.81% [nu: 0.4]
RandomForest: 84.64% [max_depth: 11] [n_estimators: 4]
KinFaceW-II fs
SVM: 79.80% [nu: 0.35]
RandomForest: 88.00% [max_depth: 14] [n_estimators: 5]
KinFaceW-II fd
SVM: 74.80% [nu: 0.5]
RandomForest: 81.80% [max_depth: 12] [n_estimators: 5]
KinFaceW-II ms
SVM: 79.80% [nu: 0.4]
RandomForest: 90.00% [max_depth: 11] [n_estimators: 5]
KinFaceW-II md
SVM: 86.40% [nu: 0.3]
RandomForest: 92.00% [max_depth: 10] [n_estimators: 4]
